In [1]:
import os
import sys
import pandas as pd
import numpy as np

In [2]:
# Add the paths to the different data source scripts to the system path
# so we can import them as modules.
apple_path = os.path.abspath('./apple_health_export')
strava_path = os.path.abspath('./strava')
samsung_path = os.path.abspath('./samsung_data')

if apple_path not in sys.path:
    sys.path.append(apple_path)
if strava_path not in sys.path:
    sys.path.append(strava_path)
if samsung_path not in sys.path:
    sys.path.append(samsung_path)

# Import the organisation and feature engineering functions from each script
try:
    from apple_health_export.organise import parse_apple_health_xml, organise_apple_health_data, engineer_activity_features as engineer_apple_features
    from strava.organise import organise_strava_data, engineer_activity_features as engineer_strava_features
    from samsung_data.organise import organise_samsung_data, engineer_activity_features as engineer_samsung_features
    print("Successfully imported all data processing modules.")
except ImportError as e:
    print(f"Error importing modules: {e}")
    print("Please ensure the 'organise.py' files exist in the correct directories.")

Successfully imported all data processing modules.


In [3]:
# --- 1. Process Apple Health Data ---
print("\n" + "="*25)
print("Processing Apple Health Data")
print("="*25)
try:
    # Apple Health requires a two-step process: parse XML, then organise from the generated CSVs
    xml_file_path = os.path.join(apple_path, 'export.xml')
    if os.path.exists(xml_file_path):
        parse_apple_health_xml(xml_file_path)
        apple_data = organise_apple_health_data(apple_path)
        for activity_id, activity_data in apple_data.items():
            if activity_data.get('track_data') is not None:
                features = engineer_apple_features(activity_data['track_data'])
                activity_data['metadata'].update(features)
        
        apple_metadata = [d['metadata'] for d in apple_data.values()]
        apple_df = pd.DataFrame(apple_metadata)
        apple_df['Source'] = 'Apple Health'
        print(f"Successfully processed {len(apple_df)} activities from Apple Health.")
    else:
        print("export.xml not found. Skipping Apple Health.")
        apple_df = pd.DataFrame()
except Exception as e:
    print(f"Could not process Apple Health data: {e}")
    apple_df = pd.DataFrame()


Processing Apple Health Data
Parsing and filtering data from export.xml...
Found 132 total workouts. After filtering, processing 17 activities.
Found 76831 heart rate records, 48873 distance records, and 100928 active energy records.
Found 103 workout routes.

Saved filtered workouts and associated health records to 'output' directory.

Reading pre-processed data from CSV files...
Successfully loaded CSV data.

Merging workouts with nearest workout routes...

Processing 17 workouts...

Processing Activity ID: 20250402_172711 (HKWorkoutActivityTypeRunning)
  - Found GPX file. Reading and cleaning...


/mnt/c/Users/agust/marathon-data/apple_health_export/organise.py:63: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pace_min_per_km'] = df['pace_min_per_km'].ffill().bfill()


  - Merging 540 heart rate points into track data.

Processing Activity ID: 20250403_083841 (HKWorkoutActivityTypeRunning)
  - Found GPX file. Reading and cleaning...


/mnt/c/Users/agust/marathon-data/apple_health_export/organise.py:63: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pace_min_per_km'] = df['pace_min_per_km'].ffill().bfill()


  - Merging 1131 heart rate points into track data.

Processing Activity ID: 20250404_102550 (HKWorkoutActivityTypeRunning)
  - Found GPX file. Reading and cleaning...
  - Merging 439 heart rate points into track data.

Processing Activity ID: 20250406_113837 (HKWorkoutActivityTypeRunning)
  - Found GPX file. Reading and cleaning...


...

/mnt/c/Users/agust/marathon-data/apple_health_export/organise.py:63: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pace_min_per_km'] = df['pace_min_per_km'].ffill().bfill()
/mnt/c/Users/agust/marathon-data/apple_health_export/organise.py:63: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pace_min_per_km'] = df['pace_min_per_km'].ffill().bfill()


  - Merging 281 heart rate points into track data.

Processing Activity ID: 20250423_120655 (HKWorkoutActivityTypeRunning)
  - Found GPX file. Reading and cleaning...
  - Merging 308 heart rate points into track data.

Processing Activity ID: 20250424_094646 (HKWorkoutActivityTypeRunning)
  - Found GPX file. Reading and cleaning...


/mnt/c/Users/agust/marathon-data/apple_health_export/organise.py:63: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pace_min_per_km'] = df['pace_min_per_km'].ffill().bfill()


  - Merging 182 heart rate points into track data.

Processing Activity ID: 20250427_094029 (HKWorkoutActivityTypeRunning)
  - Found GPX file. Reading and cleaning...
  - Merging 3397 heart rate points into track data.

Processing Activity ID: 20250505_110758 (HKWorkoutActivityTypeRunning)
  - Found GPX file. Reading and cleaning...
  - Merging 46 heart rate points into track data.

Processing Activity ID: 20250609_133258 (HKWorkoutActivityTypeRunning)
  - No route/GPX file associated with this workout.

Processing Activity ID: 20250616_140119 (HKWorkoutActivityTypeRunning)
  - No route/GPX file associated with this workout.

Processing complete.
Successfully processed 17 activities from Apple Health.


/mnt/c/Users/agust/marathon-data/apple_health_export/organise.py:63: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pace_min_per_km'] = df['pace_min_per_km'].ffill().bfill()


In [4]:
# --- 2. Process Strava Data ---
print("\n" + "="*25)
print("Processing Strava Data")
print("="*25)
try:
    strava_data = organise_strava_data(os.path.join(strava_path, 'strava-data'))
    for activity_id, activity_data in strava_data.items():
        if activity_data.get('track_data') is not None:
            # Note: Strava feature engineering might have different outputs
            features = engineer_strava_features(activity_data['track_data'])
            activity_data['metadata'].update(features)
            
    strava_metadata = [d['metadata'] for d in strava_data.values()]
    strava_df = pd.DataFrame(strava_metadata)
    strava_df['Source'] = 'Strava'
    print(f"Successfully processed {len(strava_df)} activities from Strava.")
except Exception as e:
    print(f"Could not process Strava data: {e}")
    strava_df = pd.DataFrame()


Processing Strava Data
Found 73 activities in activities.csv. Processing...

Processing Activity ID: 10024634662 (Run)
  - Expected GPX file path: /mnt/c/Users/agust/marathon-data/strava/strava-data/activities/10024634662.gpx
  - Found file: activities/10024634662.gpx. Reading and cleaning...


/mnt/c/Users/agust/marathon-data/strava/organise.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pace_min_per_km'] = df['pace_min_per_km'].ffill().bfill()
/mnt/c/Users/agust/marathon-data/strava/organise.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pace_min_per_km'] = df['pace_min_per_km'].ffill().bfill()



Processing Activity ID: 10182373857 (Run)
  - Expected GPX file path: /mnt/c/Users/agust/marathon-data/strava/strava-data/activities/10182373857.gpx
  - Found file: activities/10182373857.gpx. Reading and cleaning...

Processing Activity ID: 11233627862 (Run)
  - Expected GPX file path: /mnt/c/Users/agust/marathon-data/strava/strava-data/activities/11233627862.gpx
  - Found file: activities/11233627862.gpx. Reading and cleaning...


/mnt/c/Users/agust/marathon-data/strava/organise.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pace_min_per_km'] = df['pace_min_per_km'].ffill().bfill()



Processing Activity ID: 11285184940 (Run)
  - Expected GPX file path: /mnt/c/Users/agust/marathon-data/strava/strava-data/activities/11285184940.gpx
  - Found file: activities/11285184940.gpx. Reading and cleaning...


...

/mnt/c/Users/agust/marathon-data/strava/organise.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pace_min_per_km'] = df['pace_min_per_km'].ffill().bfill()
/mnt/c/Users/agust/marathon-data/strava/organise.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pace_min_per_km'] = df['pace_min_per_km'].ffill().bfill()



Processing Activity ID: 14261920445 (Run)
  - Expected GPX file path: /mnt/c/Users/agust/marathon-data/strava/strava-data/activities/14261920445.gpx
  - Found file: activities/14261920445.gpx. Reading and cleaning...

Processing Activity ID: 14270608835 (Run)
  - Expected GPX file path: /mnt/c/Users/agust/marathon-data/strava/strava-data/activities/14270608835.gpx
  - Found file: activities/14270608835.gpx. Reading and cleaning...


/mnt/c/Users/agust/marathon-data/strava/organise.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pace_min_per_km'] = df['pace_min_per_km'].ffill().bfill()



Processing Activity ID: 14303002789 (Run)
  - Expected GPX file path: /mnt/c/Users/agust/marathon-data/strava/strava-data/activities/14303002789.gpx
  - Found file: activities/14303002789.gpx. Reading and cleaning...

Processing complete.
Successfully processed 73 activities from Strava.


/mnt/c/Users/agust/marathon-data/strava/organise.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pace_min_per_km'] = df['pace_min_per_km'].ffill().bfill()


In [10]:
# --- 3. Process Samsung Health Data ---
print("\n" + "="*25)
print("Processing Samsung Health Data")
print("="*25)
try:
    samsung_data = organise_samsung_data(samsung_path)
    for activity_id, activity_data in samsung_data.items():
        if activity_data.get('heart_rate_data') is not None:
            features = engineer_samsung_features(activity_data['heart_rate_data'])
            activity_data['metadata'].update(features)
            
    samsung_metadata = [d['metadata'] for d in samsung_data.values()]
    samsung_df = pd.DataFrame(samsung_metadata)
    samsung_df['Source'] = 'Samsung Health'
    print(f"Successfully processed {len(samsung_df)} activities from Samsung Health.")
except Exception as e:
    print(f"Could not process Samsung Health data: {e}")
    samsung_df = pd.DataFrame()

display(samsung_df.head())


Processing Samsung Health Data
Checking file: com.samsung.health.device_profile.2025061816.csv
Checking file: com.samsung.health.exercise.2025061816.csv
Found exercise CSV: com.samsung.health.exercise.2025061816.csv
Reading heart rate data from: com.samsung.health.heart_rate.2025061816.csv
  - Found 27364 heart rate records below 90 bpm. Adjusting by +100.
Reading master exercise list from: com.samsung.health.exercise.2025061816.csv

Processing 216 running activities...

Processing Activity ID: 20191004_160055 (Running)
  - Found 132 heart rate records for this activity.

Processing Activity ID: 20191030_164032 (Running)
  - Found 29 heart rate records for this activity.

Processing Activity ID: 20190911_162329 (Running)
  - Found 100 heart rate records for this activity.

Processing Activity ID: 20191101_154416 (Running)
  - Found 33 heart rate records for this activity.

Processing Activity ID: 20190904_161928 (Running)
  - Found 50 heart rate records for this activity.

Processing 

,Activity ID,Activity Date,Activity Type,Elapsed Time,Distance,Calories,Filename,time_in_hr_zones_s,training_load_hr,avg_heart_rate_bpm,training_load_pace,pace_variability_std,time_in_pace_zones_s,Source
0,20191004_160055,2019-10-04 16:00:55+00:00,Run,1434540,2.832638,163.050003,None,"{'Z1_VeryLight': 110611.0, 'Z2_Light': 56362.0...",69999.18,163.3,0.0,0.0,{},Samsung Health
1,20191030_164032,2019-10-30 16:40:32+00:00,Run,1131590,2.175484,127.610001,None,"{'Z1_VeryLight': 0.0, 'Z2_Light': 0.0, 'Z3_Mod...",65469.72,176.1,0.0,0.0,{},Samsung Health
2,20190911_162329,2019-09-11 16:23:29+00:00,Run,1208220,2.271621,161.279999,None,"{'Z1_VeryLight': 3006.0, 'Z2_Light': 0.0, 'Z3_...",83438.53,172.1,0.0,0.0,{},Samsung Health
3,20191101_154416,2019-11-01 15:44:16+00:00,Run,1380120,2.407817,133.570007,None,"{'Z1_VeryLight': 49201.0, 'Z2_Light': 0.0, 'Z3...",56589.02,172.0,0.0,0.0,{},Samsung Health
4,20190904_161928,2019-09-04 16:19:28+00:00,Run,926740,1.843178,134.529999,None,"{'Z1_VeryLight': 209401.0, 'Z2_Light': 0.0, 'Z...",47531.98,170.9,0.0,0.0,{},Samsung Health


In [6]:
# --- 4. Standardise and Combine DataFrames ---
print("\n" + "="*25)
print("Intelligently Merging Data Sources")
print("="*25)

# Standardise column names across all dataframes BEFORE merging
# This ensures consistency when we combine them.
dataframes = {'Apple Health': apple_df, 'Strava': strava_df, 'Samsung Health': samsung_df}
for source, df in dataframes.items():
    if not df.empty:
        df.rename(columns={
            'Distance': 'Distance_km',
            'Elapsed Time': 'Duration_s',
            'training_load_hr': 'Training Load (HR)',
            'training_load_pace': 'Training Load (Pace)',
            'training_load': 'Training Load (Pace)', # For Strava
            'avg_heart_rate_bpm': 'Avg HR'
        }, inplace=True)
        # Ensure Activity Date is a datetime object for merging
        df['Activity Date'] = pd.to_datetime(df['Activity Date'], utc=True, errors='coerce', dayfirst=(source == 'Samsung Health'))

# --- Create a unified "Health" DataFrame from Apple and Samsung ---
# This will be our source for supplementary data like HR and Calories.
health_df = pd.concat([apple_df, samsung_df], ignore_index=True, sort=False)

# --- Prepare for Merging ---
# We need to sort both the primary (Strava) and supplementary (Health) data by time.
strava_df = strava_df.sort_values('Activity Date').reset_index(drop=True)
if not health_df.empty:
    health_df = health_df.sort_values('Activity Date').reset_index(drop=True)
    # Select only the columns we want to take from the health data
    health_columns_to_merge = [
        'Activity Date', 'Avg HR', 'Calories', 'Training Load (HR)', 'Source'
    ]
    # Filter to only include columns that actually exist
    health_columns_to_merge = [col for col in health_columns_to_merge if col in health_df.columns]
    health_df_for_merge = health_df[health_columns_to_merge]

# --- DIAGNOSTIC STEP: Inspect dates before merging ---
print("\n" + "="*25)
print("Pre-Merge Date Sanity Check")
print("="*25)
if not strava_df.empty:
    print("\n--- Strava DataFrame ('left' side of merge) ---")
    print(f"Date dtype: {strava_df['Activity Date'].dtype}")
    print(f"Number of null dates: {strava_df['Activity Date'].isnull().sum()}")
    if strava_df['Activity Date'].notnull().any():
        print("Date range:")
        print(f"  Start: {strava_df['Activity Date'].min()}")
        print(f"  End:   {strava_df['Activity Date'].max()}")
else:
    print("\n--- Strava DataFrame is EMPTY ---")

if 'health_df_for_merge' in locals() and not health_df_for_merge.empty:
    print("\n--- Health DataFrame ('right' side of merge) ---")
    print(f"Date dtype: {health_df_for_merge['Activity Date'].dtype}")
    print(f"Number of null dates: {health_df_for_merge['Activity Date'].isnull().sum()}")
    if health_df_for_merge['Activity Date'].notnull().any():
        print("Date range:")
        print(f"  Start: {health_df_for_merge['Activity Date'].min()}")
        print(f"  End:   {health_df_for_merge['Activity Date'].max()}")
else:
    print("\n--- Health DataFrame is EMPTY or was not created ---")
print("="*50)
# --- END OF DIAGNOSTIC STEP ---

# --- Perform the Merge ---
# We use `merge_asof` to find the closest health record for each Strava activity.
# We'll allow a small time difference (e.g., 5 minutes) to account for devices starting at slightly different times.
if not health_df.empty and not strava_df.empty:
    print("Merging Strava data with the nearest Apple/Samsung health data...")
    combined_df = pd.merge_asof(
        left=strava_df,
        right=health_df_for_merge,
        on='Activity Date',
        direction='nearest',
        tolerance=pd.Timedelta('5 minutes'),
        suffixes=('_Strava', '_Health') # Add suffixes to distinguish sources
    )
    # Rename the 'Source_Health' column to 'Health Data Source' to be clear
    if 'Source_Health' in combined_df.columns:
        combined_df.rename(columns={'Source_Health': 'Health Data Source'}, inplace=True)

else:
    print("Not enough data sources to perform a merge. Using only Strava data.")
    combined_df = strava_df.copy()


Intelligently Merging Data Sources

Pre-Merge Date Sanity Check

--- Strava DataFrame ('left' side of merge) ---
Date dtype: datetime64[ns, UTC]
Number of null dates: 0
Date range:
  Start: 2023-10-12 11:20:45+00:00
  End:   2025-04-27 09:40:01+00:00

--- Health DataFrame ('right' side of merge) ---
Date dtype: datetime64[ns, UTC]
Number of null dates: 0
Date range:
  Start: 2017-01-07 16:08:00+00:00
  End:   2025-06-16 14:01:19+00:00
Merging Strava data with the nearest Apple/Samsung health data...


In [7]:
# --- 5. Final Cleaning and Formatting ---
print("\nCleaning the final combined DataFrame...")

# Standardize 'Activity Type'
if 'Activity Type' in combined_df.columns:
    combined_df['Activity Type'] = 'Run'

# Sort by date
combined_df = combined_df.sort_values(by='Activity Date', ascending=False).reset_index(drop=True)

# Select and reorder the most important columns for the final view
final_columns = [
    'Activity Date',
    'Source', # This will be 'Strava'
    'Health Data Source', # This will be 'Apple Health' or 'Samsung Health'
    'Distance_km',
    'Duration_s',
    'Calories',
    'Avg HR',
    'Training Load (HR)',
    'Training Load (Pace)',
    'Activity ID'
]
# Filter to only include columns that actually exist in the combined dataframe
final_columns = [col for col in final_columns if col in combined_df.columns]
combined_df = combined_df[final_columns]

print("Combination and cleaning complete.")


Cleaning the final combined DataFrame...
Combination and cleaning complete.


In [8]:
# --- 6. Filter by Date Range ---
print("\n" + "="*25)
print("Filtering by Date Range")
print("="*25)

start_date_str = '2024-12-23'
end_date_str = '2025-04-28'

# Convert string dates to timezone-aware timestamps to match the 'Activity Date' column
start_date = pd.to_datetime(start_date_str, utc=True)
end_date = pd.to_datetime(end_date_str, utc=True)

print(f"Filtering activities between {start_date_str} and {end_date_str}...")

filtered_df = combined_df[
    (combined_df['Activity Date'] >= start_date) & 
    (combined_df['Activity Date'] <= end_date)
].copy()

print(f"\nFound {len(filtered_df)} activities in the specified date range.")


Filtering by Date Range
Filtering activities between 2024-12-23 and 2025-04-28...

Found 55 activities in the specified date range.


In [ ]:
# --- 7. Display Results ---
print("\n" + "="*25)
print("Combined Activity Data")
print("="*25)

print(f"\nTotal activities from all sources: {len(filtered_df)}")
print("\nDataFrame Info:")
filtered_df.info()

print("\nCombined DataFrame Head:")
display(filtered_df.head())

# print("\nCombined DataFrame Tail:")
# display(filtered_df.tail())

# print("\nActivities with Samsung Health data:")
# display(filtered_df[filtered_df['Health Data Source'] == 'Samsung Health'])

print("\nValue Counts per Source:")
display(filtered_df['Health Data Source'].value_counts())


Combined Activity Data

Total activities from all sources: 55

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 55 entries, 0 to 54
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   Activity Date         55 non-null     datetime64[ns, UTC]
 1   Health Data Source    52 non-null     object             
 2   Distance_km           55 non-null     float64            
 3   Duration_s            55 non-null     int64              
 4   Avg HR                52 non-null     float64            
 5   Training Load (HR)    52 non-null     float64            
 6   Training Load (Pace)  55 non-null     float64            
 7   Activity ID           55 non-null     int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(2), object(1)
memory usage: 3.9+ KB

Combined DataFrame Head:


,Activity Date,Health Data Source,Distance_km,Duration_s,Avg HR,Training Load (HR),Training Load (Pace),Activity ID
0,2025-04-27 09:40:01+00:00,Apple Health,42.85,17771,164.0,1094.88,733.65,14303002789
1,2025-04-24 09:46:46+00:00,Apple Health,3.01,897,166.3,56.37,54.08,14270608835
2,2025-04-23 12:06:56+00:00,Apple Health,5.00,1589,169.0,105.05,87.87,14261920445
3,2025-04-22 15:28:37+00:00,Apple Health,6.50,2172,158.6,138.17,112.78,14253317714
4,2025-04-19 13:19:54+00:00,Apple Health,18.68,7148,161.7,443.92,328.32,14224176526



Combined DataFrame Tail:


,Activity Date,Health Data Source,Distance_km,Duration_s,Avg HR,Training Load (HR),Training Load (Pace),Activity ID
50,2024-12-31 10:53:47+00:00,Samsung Health,5.01,1649,162.2,97910.63,85.37,13229927357
51,2024-12-28 13:24:54+00:00,Samsung Health,10.19,4092,164.2,229602.15,159.92,13208610584
52,2024-12-26 11:26:46+00:00,Samsung Health,5.03,1951,162.1,105399.83,85.05,13192761442
53,2024-12-25 11:17:06+00:00,Samsung Health,5.01,1775,162.2,102528.67,89.00,13186801508
54,2024-12-24 10:52:17+00:00,Samsung Health,5.09,1831,161.7,98021.60,86.72,13180366766



Activities with Samsung Health data:


,Activity Date,Health Data Source,Distance_km,Duration_s,Avg HR,Training Load (HR),Training Load (Pace),Activity ID
17,2025-03-19 19:18:56+00:00,Samsung Health,14.51,5163,164.0,0.00,226.65,13930350120
18,2025-03-18 19:12:52+00:00,Samsung Health,6.51,2100,164.2,5403.98,118.63,13919863760
19,2025-03-15 19:20:46+00:00,Samsung Health,19.00,7012,163.6,20988.23,336.58,13892848790
20,2025-03-13 19:24:24+00:00,Samsung Health,8.01,2872,164.1,30133.12,132.32,13874048926
21,2025-03-12 19:09:05+00:00,Samsung Health,13.01,4765,164.3,36873.92,229.38,13865338945
22,2025-03-11 19:08:49+00:00,Samsung Health,6.51,2254,164.7,42789.98,114.55,13855343196
23,2025-03-01 18:34:17+00:00,Samsung Health,24.00,9107,165.0,94908.12,395.48,13762810256
24,2025-02-27 18:39:44+00:00,Samsung Health,11.01,3742,165.2,107619.38,196.72,13744455872
25,2025-02-26 10:23:34+00:00,Samsung Health,5.01,1622,165.7,98598.50,91.17,13730979623
26,2025-02-25 18:48:28+00:00,Samsung Health,6.25,2486,165.1,117669.42,109.05,13725828692



Value Counts per Source:


Health Data Source
Samsung Health    38
Apple Health      14
Name: count, dtype: int64